In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import astropy.constants as c
import astropy.units  as u

## you can undo the lines below (but it will be fun for making fancy plots!)
# to make the lines below work, you have to download the file 'PostProcessingScripts' from https://github.com/FloorBroekgaarden/common_code
# and put this in the same directory as this one, and then only use the bottom line from this code below 
import sys
sys.path.append('../../common_code') 
from PostProcessingScripts import * 

In [2]:
# to make the screen wider
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Installing SSPC from source, see https://synthetic-stellar-pop-convolve.readthedocs.io/en/latest/

In [7]:
# to install uncomment the following & run this line
pip install syntheticstellarpopconvolve # is the old version 

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


### Note about installing issues that I had on MacOS & their fixes: 


Cosmic Integration SSPC 



What I had to do to make the installation from GitHub Work:


(1) distutils is deprecated in Python 3.10, so in my case it threw an error about installing this package (which happens in setup.py; 

You can fix this by doing the following (source: https://stackoverflow.com/questions/69919970/no-module-named-distutils-util-but-distutils-is-installed):

    python3 -m venv path/to/venv
    source path/to/venv/bin/activate
    python3 -m pip install xyz


(2) it throws me an error called `FileNotFoundError: [Errno 2] No such file or directory: '/Users/floorbroekgaarden/Projects/GitHub/syntheticstellarpopconvolve/dist/syntheticstellarpopconvolve-.tar.gz’` because VERSION\_NUMBER is not defined. This is because in ```instal.sh``` the “grep -oP” -P is not recognized.  This should be instead something else. But for now I fixed it by opening `install.sh` and adding a line 
```VERSION\_NUMBER=$"0.4" # fix for now ``` under VERSION_NUMBER=$(grep -oP '__version__ = "\K[^"]+' syntheticstellarpopconvolve/_version.py | awk '{print $1}') 



python3 -m venv sspc_env 
source sspc_env/bin/activate


### Example from the website

### To make this work I had to download 
```https://gitlab.com/dhendriks/syntheticstellarpopconvolve/-/blob/master/syntheticstellarpopconvolve/example_data/example_data_usecase_gw.h5```
and 

In [18]:
import os, copy, h5py
import sys
sys.path.append('/Users/floorbroekgaarden/Projects/GitHub/syntheticstellarpopconvolve/syntheticstellarpopconvolve') 


import astropy.units as u
import numpy as np
import pandas as pd
from syntheticstellarpopconvolve import convolve, default_convolution_config #, default_convolution_instruction
from syntheticstellarpopconvolve.general_functions import * # temp_dir # generate_boilerplate_outputfile,extract_unit_dict, 

TMP_DIR = temp_dir(
    "examples", "minimal_working_example", clean_path=True
)

# Create instance of output
output_hdf5_filename = os.path.join(TMP_DIR, "output_example.h5")
generate_boilerplate_outputfile(output_hdf5_filename)

# SET UP DATA
example_data = {
    "delay_time": np.array([0, 1, 2, 3]),
    "value": np.array([3, 2, 1, 0]),
    "probability": np.array([1, 2, 3, 4]),
}
example_df = pd.DataFrame.from_records(example_data)
example_df.to_hdf(output_hdf5_filename, key="input_data/example")

# Set up global configuration
convolution_config = copy.copy(default_convolution_config)
convolution_config["output_filename"] = output_hdf5_filename

# Set up SFR
convolution_config["SFR_info"] = {
    "lookback_time_bin_edges": np.array([0, 1, 2, 3, 4, 5]) * u.yr,
    "starformation_rate_array": np.array([1, 2, 3, 4, 5]) * u.Msun / u.yr
}

# set up convolution bin edges
convolution_config["convolution_lookback_time_bin_edges"] = (
    np.array([0, 1]) * u.yr
)

# Set up the convolution instructions
convolution_config["convolution_instructions"] = [
    {
        **default_convolution_instruction,
        "input_data_name": "example",
        "output_data_name": "example",
        "data_column_dict": {
            "delay_time": "delay_time",
            "normalized_yield": {"column_name": "probability", "unit": 1/u.Msun},
        },
    }
]

# run convolution
convolve(convolution_config)

# read out results
with h5py.File(
    convolution_config["output_filename"], "r"
) as output_hdf5file:
    groupname = "output_data/example/example/convolution_results/0.5 yr/"

    data = output_hdf5file[groupname + "/yield"][()]
    unit_dict = extract_unit_dict(output_hdf5file, groupname)

    print(data)

NameError: name 'generate_boilerplate_outputfile' is not defined